In [ ]:
import collections
import sys
sys.path.append("..")

import tqdm
import numpy as np
from IPython.display import SVG, HTML
from matplotlib import pyplot as plt
import tszip

import sc2ts
base_sc2_time = "2021-06-30"  # equivalent of day 0 in the sc2_ts file
sc2_ts = tszip.decompress(f"../results/upgma-full-md-30-mm-3-{base_sc2_time}.ts.tsz")


In [ ]:

times, counts = np.unique(sc2_ts.nodes_time[sc2_ts.samples()], return_counts=True)
times = times[counts > 5]  # Some times have few samples
data = {}
most_common_recombinant_mixes = {}
num_stored_recombinant_mixes = 3
for time in tqdm.tqdm(times):
    ts = sc2_ts.simplify(sc2_ts.samples(time=time), keep_unary=True, filter_nodes=False)
    recombinants_in_ancestry = {u: set() for u in ts.samples()}
    first_edge = ts.edges_left.min()
    for ed, tree in zip(ts.edge_diffs(), ts.trees(sample_lists=True)):
        if ed.interval.left == first_edge:
            continue  # Skip the first tree, for speed: recombinants arrive in intermediate trees anyway
        for u in [e.child for e in ed.edges_in if (ts.node(e.child).flags & sc2ts.NODE_IS_RECOMBINANT)]:
            for v in tree.samples(u):
                recombinants_in_ancestry[v].add(u)
    data[time] = np.bincount([len(s) for s in recombinants_in_ancestry.values()], minlength=50)
    most_common_recombinant_mix[time] = collections.Counter(
        frozenset(v) for v in recombinants_in_ancestry.values()
    ).most_common(num_stored_recombinant_mixes)

In [ ]:
most_recent = min(most_common_recombinant_mix.keys())
print(
    f"At the most recent sampling time ({most_recent}),",
    "the most common recombinant mixes are:",
)
for val, count in most_common_recombinant_mix[most_recent]:
    print(
        val,
        count,
        f"({count/np.sum(data[most_recent]) * 100}% of all samples on that day)",
    )

In [ ]:
import pandas as pd
df = pd.DataFrame(data).T
# Remove the zero padded columns
print(df.sum(axis=0))
df.drop(df.iloc[:, 24:], inplace=True, axis=1)
df

In [ ]:
df_sum = df.copy()
df_sum.iloc[:, 3] = df_sum.iloc[:, 3:].sum(axis=1)
df_sum.drop(df_sum.iloc[:, 7:], inplace=True, axis=1)
df_sum.rename(columns={0: "0", 1: "1", 2: "2", 3: "3+}, inplace=True)
pro = df_sum.div(df_sum.sum(axis=1), axis=0)
pro

In [ ]:
plt.figure(figsize=(10, 4))
p4 = plt.bar(pro.index, pro["0"],bottom=(pro["3+"] + pro["2"] + pro["1"]), width=1.0, label="0")
p3 = plt.bar(pro.index, pro["1"],bottom=(pro["3+"] + pro["2"]), width=1.0, label="1")
p2 = plt.bar(pro.index, pro["2"],bottom=pro["3+"], width=1.0, label="2")
p1 = plt.bar(pro.index, pro["3+"], width=1.0, label="3+")
plt.gca().legend(title="Number of\nrecombinants\nin ancestry")
plt.gca().invert_xaxis()
plt.xlabel(f"Days before {base_sc2_time}")
plt.ylabel(f"Daily proportion of samples")
